### Fine Tuning BERT for Regression Tasks
The original [tutorial](https://gluon-nlp.mxnet.io/master/examples/sentence_embedding/bert.html)

Download and install gluonnlp optimized model for an efficient fine tuning of nlp models.

In [ ]:
#pip install --upgrade mxnet gluonnlp

Warmup phase with import and model selection

In [ ]:
#import gluonnlp as nlp; import mxnet as mx;
#model, vocab = nlp.model.get_model('bert_12_768_12', dataset_name='wiki_multilingual_cased', use_classifier=False, use_decoder=False);
#tokenizer = nlp.data.BERTTokenizer(vocab, lower=True);
#transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=512, pair=False, pad=False);
#sample = transform(['Hello world!']);
#words, valid_len, segments = mx.nd.array([sample[0]]), mx.nd.array([sample[1]]), mx.nd.array([sample[2]]);
#seq_encoding, cls_encoding = model(words, segments, valid_len);

In [4]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
#from bert import data

nlp.utils.check_version('0.8.1')

In [5]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

In [ ]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='wiki_multilingual_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

Vocab file is not found. Downloading.
